In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv3D, BatchNormalization, MaxPool3D, Dense, Flatten, Dropout, ConvLSTM2D, LSTM
from tensorflow.keras.models import model_from_json

2024-10-16 12:06:07.696258: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 12:06:07.725331: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-16 12:06:08.237773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
pip install scikit-image

/bin/bash: /home/mca/anaconda3/envs/dse/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


##### importing libraries

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import skimage.transform
from skimage import io
from sklearn.preprocessing import OneHotEncoder
import joblib

import tensorflow as tf

#### Selecting Gestures

In [3]:
gesture_list = ['Swiping Right','Swiping Left','Thumb Up','Thumb Down','No gesture','Zooming In With Full Hand','Zooming Out With Full Hand']

#train
file_prefix = "new_jester_3DCNN"
df = pd.read_csv('./annotations/jester-v1-train.csv',header=None,names=['id','labels'])
df = df[df['labels'].isin(gesture_list)]
df.to_csv('{}_train.csv'.format(file_prefix),sep=';',index=False)
# pd.read_csv('jester-v1-8classes_train.csv',sep=";").head()
print(df.shape)
print(df.head())
#val
df = pd.read_csv('./annotations/jester-v1-validation.csv',sep=';',header=None,names=['id','labels'])
df = df[df['labels'].isin(gesture_list)]
df.to_csv('{}_val.csv'.format(file_prefix),sep=';',index=False)
print(df.shape)
print(df.head())

print('Csvs done')

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, file_path, batch_size=2, image_dim=(256,256), frames_count=36, n_channels=1, base_dir='./20bn-jester-v1/', n_classes=27,validation=False):
        self.image_dim = image_dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = True 
        self.frames_count = frames_count
        self.df = pd.read_csv(file_path,sep=";")
        self.df.id = self.df.id.map(str)
        if "train" in file_path:
            self.encoder = OneHotEncoder(sparse=False)
            self.encoder.fit(self.df.labels.values[:,None])
            joblib.dump(self.encoder,"{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))
            np.save("encoder_classes_{}_npy.npy".format(n_classes),self.encoder.categories_)
        else:
            self.encoder = joblib.load("{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))
        self.base_dir = base_dir
        self.on_epoch_end()

    def __len__(self):
        ## Decides step_size
        return self.df.shape[0] // self.batch_size

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = self.df.loc[indexes,"id"].to_list()
        X, y = self.__data_generation(indexes)
        return X, y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.df.shape[0])
        np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        X = np.empty((self.batch_size,self.frames_count, *self.image_dim, self.n_channels))
        y = np.empty((self.batch_size,1), dtype=str)
        y = []
        for i, ID in enumerate(indexes):
            files_list = self.standardize_frame_count(glob(self.base_dir + self.df.loc[ID,"id"] + "/*.jpg"),self.df.loc[ID])
            for idx,filename in enumerate(files_list):
                X[i,idx] = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(filename,color_mode='grayscale',target_size=self.image_dim))
            y.append(self.df.loc[ID,"labels"])
        encoded = self.encoder.transform(np.array(y)[:,None])
        return X,encoded
        
    def standardize_frame_count(self,files,error_check):
        shape = len(files)
        if shape < self.frames_count:
            to_add = self.frames_count - shape
            mid  = len(files)//2
            dup = [files[mid]]*to_add
            files = files[:mid] + dup + files[mid+1:]
        elif shape > self.frames_count:
            to_remove = (shape - self.frames_count)
            to_remove = int(to_remove) if int(to_remove) == to_remove else int(to_remove) + 1
            files = files[to_remove:]
        return files
    
params = {'batch_size': 56,
          'n_classes': len(gesture_list),
          'n_channels': 3,
          'image_dim': (32,32)
          }

# Generators
training_generator = DataGenerator(file_path="{}_train.csv".format(file_prefix),**params)
validation_generator = DataGenerator(file_path="{}_val.csv".format(file_prefix),**params)


(29819, 2)
        id                      labels
7   136859                    Thumb Up
8    68574               Swiping Right
13   20706                  No gesture
14   42237                  Thumb Down
19  133442  Zooming Out With Full Hand
(3640, 2)
       id                     labels
0    9223                   Thumb Up
2   42920               Swiping Left
3  106485                 Thumb Down
6   35341  Zooming In With Full Hand
7   94928              Swiping Right
Csvs done


/home/mca/anaconda3/envs/dse/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from sklearn.preprocessing import OneHotEncoder
# import joblib
# from glob import glob

# class DataGenerator(tf.keras.utils.Sequence):
#     def __init__(self, file_path, batch_size=2, image_dim=(256, 256), frames_count=36,
#                  n_channels=1, base_dir='./20bn-jester-v1/', n_classes=27, validation=False):
#         self.image_dim = image_dim
#         self.batch_size = batch_size
#         self.n_channels = n_channels
#         self.n_classes = n_classes
#         self.shuffle = True 
#         self.frames_count = frames_count
#         self.df = pd.read_csv(file_path, sep=";")
#         self.df.id = self.df.id.map(str)

#         if "train" in file_path:
#             self.encoder = OneHotEncoder(sparse_output=False)
#             self.encoder.fit(self.df.labels.values[:, None])
#             joblib.dump(self.encoder, "{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))
#             np.save("encoder_classes_{}_npy.npy".format(n_classes), self.encoder.categories_)
#         else:
#             self.encoder = joblib.load("{}_encoder_joblib.joblib".format('_'.join(file_path.split('_')[:-1])))

#         self.base_dir = base_dir
#         self.on_epoch_end()

#     def __len__(self):
#         return self.df.shape[0] // self.batch_size

#     def __getitem__(self, index):
#         indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
#         X, y = self.__data_generation(indexes)
#         return X, y 

#     def on_epoch_end(self):
#         self.indexes = np.arange(self.df.shape[0])
#         if self.shuffle:
#             np.random.shuffle(self.indexes)

#     def __data_generation(self, indexes):
#         X = np.empty((self.batch_size, self.frames_count, *self.image_dim, self.n_channels))
#         y = []
        
#         for i, ID in enumerate(indexes):
#             files_list = self.standardize_frame_count(glob(self.base_dir + self.df.loc[ID, "id"] + "/*.jpg"), self.df.loc[ID])
            
#             for idx, filename in enumerate(files_list):
#                 if filename is not None:  # Check if filename is valid
#                     X[i, idx] = tf.keras.preprocessing.image.img_to_array(
#                         tf.keras.preprocessing.image.load_img(filename, color_mode='grayscale', target_size=self.image_dim)
#                     )
#                 else:
#                     X[i, idx] = np.zeros((*self.image_dim, self.n_channels))  # Fill with zeros if no file

#             y.append(self.df.loc[ID, "labels"])

#         encoded = self.encoder.transform(np.array(y)[:, None])
#         return X, encoded
        
#     def standardize_frame_count(self, files, error_check):
#         shape = len(files)
        
#         if shape < self.frames_count:
#             to_add = self.frames_count - shape
#             if shape > 0:
#                 mid = len(files) // 2
#                 dup = [files[mid]] * to_add
#                 files = files[:mid] + dup + files[mid + 1:]
#             else:
#                 files = [None] * self.frames_count  # Handle case with no files
#         elif shape > self.frames_count:
#             to_remove = shape - self.frames_count
#             files = files[to_remove:]

#         return files

# # Parameters
# params = {
#     'batch_size': 56,
#     'n_classes': len(gesture_list),
#     'n_channels': 3,
#     'image_dim': (32, 32)
# }

# # Generators
# training_generator = DataGenerator(file_path="{}_train.csv".format(file_prefix), **params)
# validation_generator = DataGenerator(file_path="{}_val.csv".format(file_prefix), **params)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv3D, BatchNormalization, MaxPool3D, Dense, Flatten, Dropout, ConvLSTM2D, LSTM
from tensorflow.keras.models import model_from_json

In [6]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660 Ti, compute capability 7.5


2024-10-16 12:06:14.738353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 12:06:14.756893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 12:06:14.757083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
def build_model(n_classes=7):
    momentum = 0.99
    
    model = tf.keras.Sequential()
    model.add(Conv3D(64,kernel_size=3,strides=1,padding='valid',activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1,2,2),padding='same'))

    model.add(Conv3D(128,kernel_size=3,strides=1,padding='valid',activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2,2,2),padding='same'))

    model.add(Conv3D(256,kernel_size=3,strides=1,padding='valid',activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2,2,2),padding='same'))

    model.add(Conv3D(256,kernel_size=3,strides=1,padding='valid',activation='elu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2,2,2),padding='same'))
    
    model.add(Flatten())

    model.add(Dense(512,activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes,activation='softmax'))
    #batch_size,n_classes
    return model

model = build_model(n_classes=len(gesture_list))
optimizer = SGD(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['accuracy']
)

h=model.fit(training_generator,validation_data=validation_generator,validation_steps=32,epochs=50,verbose=1)

def save_model(model,file_path):
    model_json = model.to_json()
    with open(file_path+'.json', "w") as json_file:
        json_file.write(model_json)
    model.save_weights(file_path+'.h5')
save_model(model,'{}_model'.format(file_prefix))



2024-10-16 12:06:17.031631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 12:06:17.031825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 12:06:17.031903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/50
  2/532 [..............................] - ETA: 3:01 - loss: 3.0996 - accuracy: 0.1429   

2024-10-16 12:07:21.448717: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f41d40029b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-16 12:07:21.448759: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-10-16 12:07:21.509535: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


532/532 [==============================] - 238s 347ms/step - loss: 1.6425 - accuracy: 0.4551 - val_loss: 0.9992 - val_accuracy: 0.6546
Epoch 2/50
532/532 [==============================] - 181s 341ms/step - loss: 1.0391 - accuracy: 0.6310 - val_loss: 0.8823 - val_accuracy: 0.6842
Epoch 3/50
532/532 [==============================] - 182s 342ms/step - loss: 0.8360 - accuracy: 0.7034 - val_loss: 0.6868 - val_accuracy: 0.7600
Epoch 4/50
532/532 [==============================] - 176s 331ms/step - loss: 0.7153 - accuracy: 0.7481 - val_loss: 0.6489 - val_accuracy: 0.7628
Epoch 5/50
532/532 [==============================] - 177s 333ms/step - loss: 0.6403 - accuracy: 0.7736 - val_loss: 0.6343 - val_accuracy: 0.7757
Epoch 6/50
532/532 [==============================] - 177s 333ms/step - loss: 0.5657 - accuracy: 0.7983 - val_loss: 0.5476 - val_accuracy: 0.8108
Epoch 7/50
532/532 [==============================] - 182s 341ms/step - loss: 0.5091 - accuracy: 0.8206 - val_loss: 0.5984 - val_accura

In [15]:
h=model.evaluate(validation_generator)

65/65 [==============================] - 21s 317ms/step - loss: nan - accuracy: 0.1464


### issue
val loss becoming nan, after 13th epoch

If you're training for cross entropy, add a small number like 1e-8 to your output probability.

Because log(0) is negative infinity, when your model trained enough the output distribution will be very skewed, for instance say I'm doing a 4 class output, in the beginning my probability looks like

0.25 0.25 0.25 0.25
but toward the end the probability will probably look like

1.0 0 0 0
And you take a cross entropy of this distribution everything will explode. The fix is to artifitially add a small number to all the terms to prevent this.
